In [1]:
print('ok')

ok


In [6]:
# Step 1: Install dependencies
# !pip install pinecone-client transformers torch

import pinecone
from transformers import AutoTokenizer, AutoModel
import torch


In [7]:
# Load pre-trained model and tokenizer
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [10]:
def get_text_embeddings(texts, tokenizer, model):
    """
    Generate embeddings for a list of texts using Hugging Face Transformer model.
    
    Args:
        texts (list): List of strings to generate embeddings for.
        tokenizer: Tokenizer from the transformer model.
        model: Pre-trained model to generate embeddings.
    
    Returns:
        Tensor: Embeddings for the provided texts.
    """
    # Tokenize the input text
    inputs = tokenizer(
        texts, 
        padding=True, 
        truncation=True, 
        return_tensors="pt"
    )




    # Generate embeddings
    with torch.no_grad():
        model_output = model(**inputs)


    # Mean Pooling - Take the average of all token embeddings
    attention_mask = inputs['attention_mask']
    token_embeddings = model_output.last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size())
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(attention_mask.sum(1, keepdim=True), min=1e-9)
    embeddings = sum_embeddings / sum_mask
    
    return embeddings

In [11]:
texts = [
    "Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class statistical algorithms, to surpass many previous machine learning approaches in performance.",
    "ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] The application of ML to business problems is known as predictive analytics."
    "Statistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning."
    "From a theoretical viewpoint, probably approximately correct learning provides a framework for describing machine learning."

]

In [12]:
# Generate embeddings
embeddings = get_text_embeddings(texts, tokenizer, model)

In [13]:
embeddings

tensor([[-7.8979e-02, -7.9871e-02,  1.2808e-01, -1.6164e-02, -7.2941e-02,
         -5.1424e-02, -6.0325e-02, -1.3940e-01, -8.0964e-02, -1.1178e-01,
         -6.5598e-02,  1.6637e-02, -4.3588e-03, -1.7051e-01, -1.0428e-01,
          9.2779e-02,  7.6355e-02,  6.0739e-02, -2.1642e-01, -1.8161e-01,
          7.6430e-02,  7.9510e-02, -1.1186e-01,  5.8777e-02,  3.9288e-02,
          7.6145e-02,  1.9111e-01,  4.6178e-02,  6.0764e-03, -2.1116e-02,
          4.6317e-02, -1.1789e-01,  7.5416e-02,  1.3633e-01, -1.1116e-01,
          6.0456e-02, -1.2224e-01,  2.6695e-02,  6.4817e-02,  3.9606e-03,
         -7.1865e-02, -1.3902e-01,  7.8500e-02, -5.8333e-02,  1.3881e-01,
          2.1609e-01, -6.2737e-02, -2.1284e-01, -4.5482e-02,  6.5426e-02,
         -1.3553e-01, -3.9812e-02, -1.1468e-01,  6.4815e-02, -7.5684e-02,
          1.3936e-02,  4.0271e-02,  6.4993e-02,  2.6059e-02,  7.2157e-02,
          3.9550e-02, -1.7108e-01,  2.6400e-02,  8.1491e-03,  1.0496e-02,
         -1.6785e-03, -5.3882e-02,  7.

In [14]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="pcsk_iv3Ad_6QfBdBqccRwqzDNQ2CnGpFqj6GPyWVjn3wr1AKZNeHLBZMYzL3MSuKNUwN7yYqY")

In [27]:
index_name = "quickstart"

pc.create_index(
    name=index_name,
    dimension=1536, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [26]:
embeddings

tensor([[-7.8979e-02, -7.9871e-02,  1.2808e-01, -1.6164e-02, -7.2941e-02,
         -5.1424e-02, -6.0325e-02, -1.3940e-01, -8.0964e-02, -1.1178e-01,
         -6.5598e-02,  1.6637e-02, -4.3588e-03, -1.7051e-01, -1.0428e-01,
          9.2779e-02,  7.6355e-02,  6.0739e-02, -2.1642e-01, -1.8161e-01,
          7.6430e-02,  7.9510e-02, -1.1186e-01,  5.8777e-02,  3.9288e-02,
          7.6145e-02,  1.9111e-01,  4.6178e-02,  6.0764e-03, -2.1116e-02,
          4.6317e-02, -1.1789e-01,  7.5416e-02,  1.3633e-01, -1.1116e-01,
          6.0456e-02, -1.2224e-01,  2.6695e-02,  6.4817e-02,  3.9606e-03,
         -7.1865e-02, -1.3902e-01,  7.8500e-02, -5.8333e-02,  1.3881e-01,
          2.1609e-01, -6.2737e-02, -2.1284e-01, -4.5482e-02,  6.5426e-02,
         -1.3553e-01, -3.9812e-02, -1.1468e-01,  6.4815e-02, -7.5684e-02,
          1.3936e-02,  4.0271e-02,  6.4993e-02,  2.6059e-02,  7.2157e-02,
          3.9550e-02, -1.7108e-01,  2.6400e-02,  8.1491e-03,  1.0496e-02,
         -1.6785e-03, -5.3882e-02,  7.

In [28]:

tensor = torch.tensor([[-7.8979e-02, -7.9871e-02,  1.2808e-01, -1.6164e-02, -7.2941e-02,
         -5.1424e-02, -6.0325e-02, -1.3940e-01, -8.0964e-02, -1.1178e-01,
         -6.5598e-02,  1.6637e-02, -4.3588e-03, -1.7051e-01, -1.0428e-01,
          9.2779e-02,  7.6355e-02,  6.0739e-02, -2.1642e-01, -1.8161e-01,
          7.6430e-02,  7.9510e-02, -1.1186e-01,  5.8777e-02,  3.9288e-02,
          7.6145e-02,  1.9111e-01,  4.6178e-02,  6.0764e-03, -2.1116e-02,
          4.6317e-02, -1.1789e-01,  7.5416e-02,  1.3633e-01, -1.1116e-01,
          6.0456e-02, -1.2224e-01,  2.6695e-02,  6.4817e-02,  3.9606e-03,
         -7.1865e-02, -1.3902e-01,  7.8500e-02, -5.8333e-02,  1.3881e-01,
          2.1609e-01, -6.2737e-02, -2.1284e-01, -4.5482e-02,  6.5426e-02,
         -1.3553e-01, -3.9812e-02, -1.1468e-01,  6.4815e-02, -7.5684e-02,
          1.3936e-02,  4.0271e-02,  6.4993e-02,  2.6059e-02,  7.2157e-02,
          3.9550e-02, -1.7108e-01,  2.6400e-02,  8.1491e-03,  1.0496e-02,
         -1.6785e-03, -5.3882e-02,  7.1424e-02,  3.4894e-02, -1.0190e-01,
          1.0264e-03,  1.6788e-01, -7.2952e-02,  1.5089e-01, -1.2079e-02,
         -1.7750e-02,  6.8399e-02, -1.4752e-01,  1.3017e-01,  4.8779e-02,
          5.4911e-02,  8.3173e-03,  3.2668e-02,  1.5339e-01,  2.1173e-01,
         -1.2276e-01, -6.4467e-02,  6.1128e-02, -6.0154e-02, -1.2393e-01,
         -1.2310e-01, -1.0183e-01, -9.3804e-02, -7.4582e-02, -6.6876e-02,
          6.6307e-02, -4.0174e-02, -2.1128e-01, -8.9185e-03,  1.3065e-01,
         -1.1938e-01,  1.0961e-01, -4.8376e-02,  1.5786e-02,  1.1642e-01,
         -3.7845e-02,  1.2889e-01, -1.4174e-02,  1.3318e-01, -2.2535e-01,
          4.7881e-02, -4.7066e-02, -8.9322e-02,  1.0270e-02, -3.3961e-02,
         -1.7203e-01, -8.8847e-02, -8.6836e-03, -8.6213e-02,  2.2218e-01,
         -6.2598e-02,  7.9950e-02,  2.0015e-03,  2.4260e-02, -3.9837e-02,
         -5.9290e-02, -1.5954e-01,  1.0200e-32, -1.2544e-01, -1.2588e-01,
         -3.0754e-02,  2.4222e-02,  4.6828e-02, -1.0366e-01, -6.2185e-02,
         -7.9069e-02,  1.1456e-01,  7.5290e-02, -6.4930e-02, -1.0764e-01,
         -3.7418e-02,  1.6091e-01,  6.1135e-02,  9.1764e-02,  6.7668e-02,
          5.7694e-02, -5.5515e-02, -1.1600e-01,  8.7849e-02,  1.1304e-01,
          1.1464e-01, -4.8612e-02, -3.4035e-02,  1.2191e-01,  1.0533e-01,
         -1.3996e-02,  6.5925e-02,  1.5921e-03,  1.2898e-02,  2.0603e-02,
         -1.0603e-01, -1.1015e-02,  1.1954e-01, -1.5925e-02, -6.3703e-03,
          5.6951e-02,  4.7910e-02,  8.4670e-02, -9.5760e-02, -1.7095e-02,
          1.3335e-01, -6.0566e-02, -1.0354e-01,  4.9475e-04, -1.7307e-02,
         -1.2546e-01, -3.1737e-02, -1.5980e-01,  1.2118e-02, -1.2672e-01,
         -1.4494e-01, -3.0498e-02,  7.3884e-02,  8.6767e-02,  7.4433e-03,
          8.6276e-02, -8.4744e-02,  5.4252e-02, -2.1616e-02, -1.0406e-02,
          1.5940e-02,  1.3535e-01,  8.3261e-04,  1.0487e-01,  9.8363e-02,
          1.8461e-01,  1.1149e-01, -6.9705e-02,  1.2357e-01,  7.0819e-02,
          6.9014e-02, -3.2277e-02, -2.2520e-02,  5.7902e-02,  7.9803e-02,
         -9.5307e-02,  9.2092e-03,  1.2549e-01, -4.6797e-02,  2.6374e-02,
         -2.0157e-02, -1.2694e-01, -2.8990e-02,  7.2010e-02, -3.0731e-02,
         -8.6674e-02, -5.8608e-02,  9.0073e-03, -2.5526e-01, -1.9316e-02,
         -3.3391e-02,  5.3369e-02, -6.3179e-02, -1.3799e-32, -1.0786e-01,
          1.6395e-01, -5.9122e-02,  1.5720e-01,  1.5579e-02, -5.4912e-03,
         -1.5828e-01, -1.2288e-02, -6.0997e-02,  1.6723e-01,  6.5034e-03,
          3.0696e-02,  7.9454e-02,  6.2118e-02, -6.3782e-02,  1.1832e-01,
         -1.3203e-01, -6.2498e-02, -7.1769e-02, -6.0618e-03, -1.4011e-01,
          1.7817e-01,  7.0404e-02, -9.6291e-02,  6.0730e-02, -1.0519e-01,
         -1.2060e-01,  1.0812e-01, -4.2261e-03, -3.6233e-02, -6.6643e-02,
         -4.9577e-02, -7.8398e-02, -1.7393e-02,  3.2587e-02,  1.3875e-01,
          1.4410e-01, -6.0377e-02, -1.3447e-02,  3.2101e-02,  9.6474e-02,
          1.2062e-02, -7.4047e-02, -1.5173e-01, -5.3450e-02, -8.2791e-02,
         -6.6925e-02,  3.7779e-02,  1.2759e-01, -7.4096e-02, -1.8290e-03,
         -2.6135e-02, -5.3869e-02,  2.0914e-02, -4.1067e-02,  6.5749e-02,
         -4.1200e-02,  4.6806e-02,  8.8177e-02,  2.4715e-01, -9.4186e-02,
         -1.7103e-01,  2.2561e-02,  1.0238e-01, -5.5040e-02,  1.3204e-01,
         -3.6563e-02,  1.5258e-01, -1.9816e-02,  2.0769e-02,  8.8217e-02,
          8.0032e-02,  2.9341e-02,  1.6895e-01, -1.7122e-01, -1.2660e-01,
          7.0628e-03, -1.2572e-01,  3.3091e-03, -3.8012e-02,  1.2714e-01,
         -2.8434e-01, -3.5150e-02,  6.9108e-02,  1.4185e-01, -4.4249e-02,
          6.0749e-02, -2.2756e-02,  5.8363e-02, -1.2699e-01, -3.1537e-02,
          1.4957e-01, -1.1919e-01, -4.7368e-02, -1.8706e-01, -9.8349e-08,
         -2.2333e-02, -3.4408e-03,  2.0626e-01, -8.2787e-02,  2.3173e-01,
          6.5498e-02, -6.8726e-02,  1.6137e-01, -5.9205e-02, -7.6706e-02,
          1.8955e-01, -8.1862e-02, -8.6370e-02, -3.8517e-02,  1.5560e-02,
          9.0583e-02,  7.4012e-02,  4.2698e-02,  8.6106e-02,  1.7421e-02,
          1.0641e-01, -8.0690e-02,  1.3008e-01, -3.0859e-02,  7.5455e-02,
         -2.1669e-01, -8.6606e-02,  8.2387e-02, -3.7134e-02,  2.0640e-02,
         -1.4902e-01,  1.0067e-01,  8.0706e-02, -3.4520e-02,  1.4211e-01,
          2.7022e-01,  1.2337e-01, -1.2064e-01, -7.2106e-02, -9.5107e-02,
         -9.3593e-02,  1.6288e-01, -1.2526e-01, -2.6978e-02,  3.5783e-03,
          2.9031e-02,  1.5806e-01, -1.5180e-01,  2.7388e-02,  5.7060e-02,
          1.1040e-01,  1.5296e-02,  1.9851e-01,  1.4056e-01,  1.0429e-01,
          1.1399e-01,  7.9995e-02, -6.6441e-02,  1.8460e-02,  1.0784e-01,
          4.0643e-02,  7.5225e-02, -5.8863e-02, -1.1608e-01],
        [-1.8619e-01, -7.5679e-03,  9.0708e-03, -2.6405e-03,  6.0003e-02,
         -6.0053e-02,  4.9973e-02, -7.2424e-02, -1.9557e-01,  2.4244e-02,
         -6.1621e-02,  5.2262e-02,  8.5601e-02,  1.1761e-02, -1.4692e-01,
          1.1273e-01,  8.9322e-02,  1.1254e-01, -1.8643e-01, -3.5901e-01,
         -1.3933e-01,  7.9164e-02, -1.8084e-01,  4.7661e-02,  5.1259e-03,
          9.5615e-02,  2.3283e-01,  4.7475e-02, -4.5949e-02, -6.7967e-02,
          1.0094e-01,  4.3883e-02,  1.1566e-01,  1.6699e-02, -1.9757e-01,
          1.1931e-01, -1.6123e-01,  9.9678e-02,  9.0922e-02,  3.8460e-02,
         -1.1390e-01, -1.5802e-01, -1.0197e-01, -5.1612e-02,  2.4200e-01,
          1.6060e-02, -2.2124e-01, -7.5373e-02, -2.9501e-02,  5.4428e-02,
         -2.5516e-01, -1.2030e-01,  2.3098e-02, -1.4041e-02, -5.1639e-02,
         -6.2325e-02, -8.3909e-03, -2.6633e-02,  1.6263e-01,  2.0961e-02,
          1.0486e-01, -2.4327e-01,  4.8442e-02,  5.3375e-03,  1.1903e-02,
         -1.3150e-02, -2.7891e-02, -5.1528e-03, -1.3975e-01, -5.1968e-02,
         -1.1046e-02,  2.6622e-01, -2.5880e-01,  2.1742e-01,  6.7338e-02,
          1.2993e-01,  1.5690e-01, -8.7104e-02,  1.6741e-01,  4.7667e-02,
         -2.3044e-03,  2.2109e-01, -1.0170e-01,  1.0638e-01,  1.3823e-01,
         -1.7143e-01, -6.0111e-02,  5.4421e-02,  1.9807e-01,  1.3466e-02,
         -8.2953e-02, -1.2635e-01, -1.0363e-01, -2.4786e-02, -5.5948e-02,
          1.7050e-02, -1.7346e-01, -2.4888e-01,  1.2518e-01,  1.2833e-01,
         -1.7005e-01,  3.0672e-01,  1.6443e-02, -3.6026e-02, -6.2158e-02,
         -2.1279e-01,  2.2890e-01,  1.4163e-01,  4.5771e-02, -2.0411e-01,
          2.1449e-02, -1.6135e-01, -3.3525e-02,  3.0263e-03,  5.6267e-02,
         -2.5228e-01,  3.0121e-02,  3.1424e-02, -1.0100e-01,  2.4276e-01,
         -6.9222e-02,  1.4195e-01,  6.9635e-02,  1.2929e-02,  2.6918e-02,
         -9.3612e-02, -2.7455e-01,  3.5071e-33, -1.7316e-01, -2.0157e-01,
          2.8831e-02, -5.5614e-03,  6.2936e-02, -7.1409e-02, -1.4630e-01,
         -6.8166e-03,  2.6951e-01,  8.3139e-02,  1.0265e-03,  2.1850e-01,
          4.2421e-02,  2.6144e-01,  8.0992e-02,  7.0594e-02,  1.6764e-01,
          1.7832e-01, -1.0218e-01, -2.3280e-01,  5.1128e-02, -1.7595e-01,
          3.7667e-02, -9.6746e-02, -5.0097e-02,  1.2242e-01,  1.0983e-01,
          1.0119e-01,  2.3766e-01,  2.7516e-02,  1.6922e-01, -1.3218e-02,
         -1.2012e-01,  9.0741e-02,  1.4241e-01,  2.8167e-03,  6.8468e-02,
          2.6828e-02,  1.6675e-01,  1.0647e-01, -1.5582e-01,  5.1851e-02,
          2.2913e-01, -8.6621e-02, -2.8669e-01,  1.1694e-01,  2.8662e-02,
         -1.7265e-01,  5.3488e-02, -1.8432e-01,  8.0442e-02, -1.7077e-01,
         -3.2005e-02, -1.4746e-02, -1.3555e-01,  2.4407e-01, -1.0081e-01,
         -1.0455e-01,  7.5034e-02, -6.4077e-02, -1.9102e-01,  1.3411e-03,
          9.4813e-02, -3.7276e-02, -7.2789e-02, -5.2842e-02,  1.0359e-01,
          9.4836e-02,  1.7148e-02,  2.7213e-02,  2.0625e-02,  8.3556e-02,
          1.0789e-01,  1.1288e-02,  2.5424e-02, -8.2858e-03,  1.5077e-01,
          6.0803e-02, -1.7489e-02,  1.6562e-01, -5.1337e-02, -6.1681e-03,
         -8.5626e-03, -1.9578e-01, -4.4841e-02, -3.4407e-02,  5.2636e-02,
         -1.1327e-02, -6.9160e-02, -7.5731e-02, -3.0332e-01,  2.2126e-01,
         -1.7751e-01,  1.0171e-01,  1.3449e-01, -8.6575e-33, -1.6968e-01,
          1.6122e-01,  1.1583e-01,  3.1107e-02,  1.2174e-02,  5.5249e-02,
         -1.6827e-01,  1.4821e-02,  1.2398e-01,  2.7571e-02, -3.4676e-02,
         -1.5897e-01, -2.1630e-02,  5.7166e-02, -7.4967e-02,  1.5922e-01,
         -1.1367e-02, -7.3312e-02, -1.1570e-01,  9.5380e-02, -2.2430e-01,
          7.9631e-04, -1.3349e-01, -2.2570e-01,  3.8471e-02,  7.3336e-02,
         -1.6868e-01,  7.6640e-02, -1.1032e-01, -2.1453e-02, -9.1253e-02,
         -1.0841e-01, -1.4775e-01, -4.5248e-02, -1.7210e-01,  8.3228e-03,
          2.3323e-02, -1.6401e-01,  2.3556e-01,  1.4991e-01,  1.6686e-01,
          7.3363e-02, -9.8148e-02, -1.4358e-02,  4.4703e-02, -1.6262e-01,
          5.4866e-02,  4.3690e-03,  2.3465e-01, -2.0034e-02, -5.2101e-02,
          1.6998e-01, -1.4089e-02,  3.1184e-02, -7.2047e-02,  1.4344e-01,
          4.0236e-02, -1.1457e-01, -3.4749e-02,  2.1914e-01, -8.8115e-02,
         -6.9844e-02,  1.8900e-01,  1.5818e-01, -6.4812e-02, -4.5227e-02,
          4.4261e-02,  6.8280e-02, -1.1718e-01, -1.2238e-01,  1.0343e-01,
         -2.6687e-02, -8.6423e-02,  8.1410e-02, -2.3373e-01, -1.3676e-01,
         -4.6917e-02, -2.4379e-02,  7.5943e-03, -3.2470e-02,  2.6956e-01,
         -2.0047e-01,  6.0013e-02,  4.1599e-02,  9.6518e-02, -1.5116e-01,
          2.2999e-01, -1.1802e-01,  1.7704e-02, -1.9399e-01, -2.1745e-01,
         -4.4335e-02, -3.4526e-03,  6.9503e-02, -4.0019e-02, -9.7940e-08,
         -1.1708e-01, -1.6540e-01,  2.0776e-01, -5.8466e-02,  1.8059e-01,
         -9.6510e-03, -1.0997e-01,  3.0177e-01, -1.0745e-01, -1.0576e-01,
          8.1339e-02, -1.2609e-01, -2.2106e-01,  6.3066e-02, -8.5110e-03,
          8.3569e-02,  1.6035e-01,  1.1746e-01,  4.9363e-03,  2.4560e-01,
          7.4902e-02, -1.4769e-02,  3.6357e-02,  1.7070e-02,  1.2953e-01,
         -1.2169e-01,  2.4649e-02,  1.3807e-01, -3.4363e-02, -4.0230e-04,
         -1.3489e-01,  1.2835e-01,  1.2838e-01, -6.2349e-02,  9.1546e-02,
          1.1806e-01,  1.7160e-01, -3.0680e-01, -2.7332e-01, -2.6095e-01,
         -3.4063e-02,  2.7067e-01, -9.2734e-02, -3.1933e-02,  4.5833e-02,
          3.9904e-02,  1.7714e-01, -8.3876e-02,  4.1132e-02,  5.5243e-02,
          8.3828e-02,  7.3931e-02,  2.5842e-01,  5.3983e-02,  1.6855e-01,
          1.5464e-01,  2.4129e-02, -1.4163e-01,  6.1752e-03,  9.5272e-02,
         -4.6769e-02,  6.6238e-02, -9.6485e-02, -3.4220e-02]])




In [29]:
# Convert PyTorch tensor to NumPy array
numpy_array = tensor.cpu().detach().numpy()

In [30]:
print(numpy_array)


[[-7.8979e-02 -7.9871e-02  1.2808e-01 -1.6164e-02 -7.2941e-02 -5.1424e-02
  -6.0325e-02 -1.3940e-01 -8.0964e-02 -1.1178e-01 -6.5598e-02  1.6637e-02
  -4.3588e-03 -1.7051e-01 -1.0428e-01  9.2779e-02  7.6355e-02  6.0739e-02
  -2.1642e-01 -1.8161e-01  7.6430e-02  7.9510e-02 -1.1186e-01  5.8777e-02
   3.9288e-02  7.6145e-02  1.9111e-01  4.6178e-02  6.0764e-03 -2.1116e-02
   4.6317e-02 -1.1789e-01  7.5416e-02  1.3633e-01 -1.1116e-01  6.0456e-02
  -1.2224e-01  2.6695e-02  6.4817e-02  3.9606e-03 -7.1865e-02 -1.3902e-01
   7.8500e-02 -5.8333e-02  1.3881e-01  2.1609e-01 -6.2737e-02 -2.1284e-01
  -4.5482e-02  6.5426e-02 -1.3553e-01 -3.9812e-02 -1.1468e-01  6.4815e-02
  -7.5684e-02  1.3936e-02  4.0271e-02  6.4993e-02  2.6059e-02  7.2157e-02
   3.9550e-02 -1.7108e-01  2.6400e-02  8.1491e-03  1.0496e-02 -1.6785e-03
  -5.3882e-02  7.1424e-02  3.4894e-02 -1.0190e-01  1.0264e-03  1.6788e-01
  -7.2952e-02  1.5089e-01 -1.2079e-02 -1.7750e-02  6.8399e-02 -1.4752e-01
   1.3017e-01  4.8779e-02  5.4911e-02 